In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification, BertConfig, AutoModelForSequenceClassification
from bertviz import model_view
from models import EnsembleBERT
from config import args
import torch

args.rank=0
args.target_model_path = '/pub/data/huangpei/PAT-AAAI23/TextFooler/models/bert/mr'
args.num_models = 3

tokenizer = AutoTokenizer.from_pretrained(args.target_model_path)
config = BertConfig.from_pretrained(args.target_model_path)
config.aux_weight = args.aux_weight
config.num_models = args.num_models
config.perturb_attention = args.perturb_attention
checkpoint = torch.load(args.target_model_path + '/pytorch_model.bin')
model = EnsembleBERT.from_pretrained('bert-base-uncased', state_dict=checkpoint, config=config).cuda(args.rank)    

sentence = "The cat sat on the mat"    
outs = tokenizer.encode_plus(sentence)
input_ids = outs['input_ids']
input_mask = outs['attention_mask']
if args.target_model == 'bert':
    segment_ids = outs['token_type_ids']
outputs = model(input_ids=torch.LongTensor([input_ids]).cuda(args.rank), token_type_ids=torch.LongTensor([segment_ids]).cuda(args.rank), attention_mask=torch.LongTensor([input_mask]).cuda(args.rank), output_attentions=True)
attention = outputs['attentions']  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(input_ids) 

model_view(attention, tokens)